In this notebook we will use US Census Bureau's API to get the zip level data for our economic variables. We will use the following variables:

1. Median Household Income
2. Poverty Rate
3. Unemployment Rate


In [2]:
import requests
import pandas as pd

api_key = '118733171a8f1248ede31f85f1436d2f78d14d94'

# Define the parameters for the API request
params = {
    'get': 'NAME,B17001_001E,B17001_002E,B23025_001E,B23025_005E,B19013_001E',
    'for': 'zip code tabulation area:*',
    'in': 'state:36',
    'key': api_key
}

In [3]:

# Make the API request to the ACS 5-Year Estimates endpoint
response = requests.get('https://api.census.gov/data/2019/acs/acs5', params=params)

In [4]:
# Load the response data into a DataFrame
data = response.json()
df = pd.DataFrame(data[1:], columns=['NAME', 'B17001_001E', 'B17001_002E', 'B23025_001E', 'B23025_005E', 'B19013_001E', 'state', 'zcta'])

B17001_001E is the total population for whom poverty status is determined. It is used as a denominator for the poverty rate.

B17001_002E is the total population for whom poverty status is determined and who are below poverty level. It is used as a numerator for the poverty rate.

B23025_001E is the total population for whom employment status is determined. It is used as a denominator for the unemployment rate.

B23025_005E is the total population for whom employment status is determined and who are unemployed. It is used as a numerator for the unemployment rate.

In [5]:
# Calculate the poverty and unemployment rates
df['poverty_rate'] = (df['B17001_002E'].astype(float) / df['B17001_001E'].astype(float)) * 100
df['unemployment_rate'] = (df['B23025_005E'].astype(float) / df['B23025_001E'].astype(float)) * 100


In [6]:
# Convert median household income to float
df['median_household_income'] = df['B19013_001E'].astype(float)

In [7]:
# Select only the desired columns
df = df[['zcta', 'poverty_rate', 'unemployment_rate', 'median_household_income']]

In [8]:
# Display the result
print(df.head())


    zcta  poverty_rate  unemployment_rate  median_household_income
0  11804      4.174695           0.976668                 112946.0
1  11930      4.081633           0.000000                 102639.0
2  11941      4.117974           0.000000                 123125.0
3  11962      0.000000           0.000000                 113182.0
4  11702      4.156929           2.287113                 130337.0


In [9]:
df

,zcta,poverty_rate,unemployment_rate,median_household_income
0,11804,4.174695,0.976668,112946.0
1,11930,4.081633,0.000000,102639.0
2,11941,4.117974,0.000000,123125.0
3,11962,0.000000,0.000000,113182.0
4,11702,4.156929,2.287113,130337.0
...,...,...,...,...
1789,13305,0.000000,12.500000,48387.0
1790,13321,20.879121,1.468429,50373.0
1791,13345,5.384615,13.333333,51429.0
1792,13352,0.000000,0.000000,-666666666.0


In [10]:
df.to_csv('census_data-by-zipcode.csv')